In [ ]:
import numpy as np
import cv2
import trimesh
import matplotlib.pyplot as plt
import os
import sys

parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
def draw_model_overlay(img, projected_pts, faces=None, color=(0, 255, 0)):
    img_overlay = img.copy()
    
    if faces is not None:
        # Draw mesh faces
        for face in faces:
            pts = projected_pts[face].astype(np.int32)
            cv2.polylines(img_overlay, [pts], isClosed=True, color=color, thickness=1)
    else:
        # Draw points only
        for (x, y) in projected_pts.astype(np.int32):
            cv2.circle(img_overlay, (x, y), 2, color, -1)

    return img_overlay

In [ ]:
def visualize_pose(img, model_path, R, t, K):
    mesh = trimesh.load(model_path)

    # Optional: scale vertices if model was in millimeters
    vertices = mesh.vertices.copy()
    if vertices.max() > 10:  # crude check if it's not already in meters
        vertices *= 0.001  # convert mm → meters

    faces = mesh.faces if hasattr(mesh, 'faces') else None

    projected_pts = project_3D_points_to_2D(vertices, R, t, K)
    overlay = draw_model_overlay(img, projected_pts, faces=faces)

    plt.figure(figsize=(8, 8))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()